In [93]:
import sqlite3
import numpy as np
import requests
import os
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

In [94]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [95]:
load_dotenv()

True

In [96]:
def cosine_similarity(a, b):
    
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [97]:
query_text = "Python is what type of language?"
db_path='chunks.db' 
top_k=3
min_sim=0.5

In [98]:
query_emb = model.encode([query_text])[0]

In [99]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute('SELECT chunk_id, chunk_text, embedding_vector, document_name FROM chunks')
results = c.fetchall()
conn.close()

In [100]:
results

[('sample1.txt_chunk_1',
  'Machine learning is a subset of artificial intelligence. It enables computers to learn from data without explicit programming. This field has revolutionized industries like healthcare and finance.',
  b'x\xdd\x96\xbbX\xe6C<\x85m\xc3\xbc\xc8\xd2\x85\xbb\x0b\xab \xba\x8b\x85:\xbbiKL\xbc3\x91\xa9\xba{\xa2\xb0\xbc;y\xa8\xba\x98B=\xbd\xb4"\x8d=\x87\xb0k<8\x99E\xbdH\x02b\xbce\xedB;\xef\xa3\x04\xbd\x7f\xaf\xee\xbc\xf3v\x10\xbd\xc1\xe9Q\xbd\xbfH\x18\xbc\x1b1L\xbb\xdaH)\xbdd\xd0T\xbc>\x97\x9e<\x94\xcft=\xf9\x84X\xbc\xab*7<\xe7#\x9c\xbc\xab:\x19\xb9\xb0\xc8\xcc;\xfb\x9dh=l\xee:=\x85\x7f\x1a=\xdc\xa9\x17\xbdc\xe6\xb6<\x05\x0f.<+\x9c\'=W\x0e\x94;\x8f\x00\xae\xbb\xcb\xc5@\xbd\x1d\xfe\x17\xbd\xedm@\xbc\x8b}s;\xe4\x18\x8e=\x94"\xf1<\xd5\x90K\xbd\x89\x91\x92\xbd+L\xf9\xba\x93\xec}\xbcH\xbe\xce\xbd\xf5\x9dn\xbc\x95\xe5\xaa\xbb\xb8#?<C\xa0\xac\xbc\xf5\xea\x0c=\xaf\xc2\xa7<\x93\xc4\xae;]\xb1\x11\xbd>\x82\x1e\xbc\xc4t+\xbc\x8aF\x94\xbd%\xf2\x9e\xbb\xcb\x15`=\xb7\x8fm=U\xd5s8\xc

In [101]:
similarities = []

In [102]:
for chunk_id, text, emb_blob, document_name in results:
    stored_emb = np.frombuffer(emb_blob, dtype=np.float32)
    sim = cosine_similarity(query_emb, stored_emb)
    if sim >= min_sim:
        similarities.append((sim, chunk_id, text))

In [103]:
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]

In [104]:
top_chunks

[(np.float32(0.793778),
  'sample3.txt_chunk_1',
  'Python is a high level programming language. It mostly has english like syntax.'),
 (np.float32(0.6170326),
  'sample2.txt_chunk_1',
  'Python is a versatile programming language. It is widely used for web development and data analysis.')]

In [105]:
context = "\n".join([f"- {chunk[2]}" for chunk in top_chunks]) 
prompt = f"""You are a helpful assistant. Answer the question based ONLY on the provided context below. 
Do not use any external knowledge. If the answer is not in the context, respond exactly: "I don’t know based on the provided context."
Context:{context}
Question: {query_text}
Answer:"""

In [106]:
model_name="meta-llama/Llama-3.1-8B-Instruct:fastest"

In [107]:
API_URL = "https://router.huggingface.co/v1/chat/completions" 
headers = {"Authorization": f"Bearer {os.getenv('HF_TOKEN')}", "Content-Type": "application/json"}
messages = [{"role": "user", "content": prompt}] 
payload = {
    "model": model_name,
    "messages": messages,
    "max_tokens": 150,  
    "temperature": 0.1 
}

In [108]:
response = requests.post(API_URL, headers=headers, json=payload)
if response.status_code == 200:
    result = response.json()
    responseans =  result['choices'][0]['message']['content'].strip()  
else:
    responseans = f"API Error: {response.status_code} - {response.text}"

In [109]:
if not top_chunks:
    print("No relevant chunks found. I don’t know based on the provided context.")

In [110]:
sources = [f"Chunk {chunk[1]} (sim: {chunk[0]:.2f})" for chunk in top_chunks]

In [111]:
print(f"{responseans}\n\nSources:\n" + "\n".join(sources))

Python is a high-level programming language.

Sources:
Chunk sample3.txt_chunk_1 (sim: 0.79)
Chunk sample2.txt_chunk_1 (sim: 0.62)
